In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import cifar10

In [4]:
(x_trainval, y_trainval), (x_test, y_test) = cifar10.load_data()

print('Train shape:', x_trainval.shape, y_trainval.shape)
print('Test shape:', x_test.shape, y_test.shape)

Train shape: (50000, 32, 32, 3) (50000, 1)
Test shape: (10000, 32, 32, 3) (10000, 1)


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    x_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)

print(f"Train: {x_train.shape}, Val: {x_val.shape}, Test: {x_test.shape}")

Train: (40000, 32, 32, 3), Val: (10000, 32, 32, 3), Test: (10000, 32, 32, 3)


In [6]:
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense

def build_model(input_shape, num_conv=3, filters=[32,64,128], kernel_sizes=[(3,3)]*3, pooling_type='max'):
    model = Sequential()
    for i in range(num_conv):
        if i == 0:
            model.add(Conv2D(filters[i], kernel_sizes[i], activation='relu', input_shape=input_shape, padding='same'))
        else:
            model.add(Conv2D(filters[i], kernel_sizes[i], activation='relu', padding='same'))
        if pooling_type == 'max':
            model.add(MaxPooling2D((2,2)))
        else:
            model.add(AveragePooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model


In [8]:
import os
import random
import numpy as np
import tensorflow as tf

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

from tensorflow.keras.optimizers import Adam

model = build_model(
    input_shape=(32,32,3),
    num_conv=2,                 
    filters=[32,64],       
    kernel_sizes=[(3,3)]*2,    
    pooling_type='max'          
)

model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, y_val),
    verbose=2
)

model.save_weights('weights_compare.weights.h5')


Epoch 1/10


/Users/williamglory/Desktop/IF SEM 6/ML/Tubes-2-IF3270/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-29 11:57:20.343341: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newe

625/625 - 9s - 15ms/step - accuracy: 0.4535 - loss: 1.5292 - val_accuracy: 0.5665 - val_loss: 1.2405
Epoch 2/10
625/625 - 8s - 13ms/step - accuracy: 0.5961 - loss: 1.1493 - val_accuracy: 0.6225 - val_loss: 1.0831
Epoch 3/10
625/625 - 8s - 13ms/step - accuracy: 0.6466 - loss: 1.0125 - val_accuracy: 0.6438 - val_loss: 1.0307
Epoch 4/10
625/625 - 8s - 13ms/step - accuracy: 0.6772 - loss: 0.9248 - val_accuracy: 0.6562 - val_loss: 0.9996
Epoch 5/10
625/625 - 8s - 13ms/step - accuracy: 0.7017 - loss: 0.8574 - val_accuracy: 0.6654 - val_loss: 0.9853
Epoch 6/10
625/625 - 9s - 14ms/step - accuracy: 0.7222 - loss: 0.8018 - val_accuracy: 0.6731 - val_loss: 0.9679
Epoch 7/10
625/625 - 9s - 14ms/step - accuracy: 0.7405 - loss: 0.7507 - val_accuracy: 0.6859 - val_loss: 0.9476
Epoch 8/10
625/625 - 9s - 14ms/step - accuracy: 0.7574 - loss: 0.7044 - val_accuracy: 0.6861 - val_loss: 0.9451
Epoch 9/10
625/625 - 9s - 14ms/step - accuracy: 0.7721 - loss: 0.6612 - val_accuracy: 0.6866 - val_loss: 0.9569
Epo

In [9]:
import numpy as np
from sklearn.metrics import f1_score
from CNNScratch import CNNScratch  

keras_model = build_model(
    input_shape=(32,32,3),
    num_conv=2,                       
    filters=[32,64],        
    kernel_sizes=[(3,3), (3,3)],
    pooling_type='max'
)
keras_model.load_weights('weights_compare.weights.h5')  

scratch_model = CNNScratch(keras_model)

batch_size = 32
y_pred_scratch_prob = scratch_model.forward(x_test, batch_size=batch_size)  
y_pred_scratch = np.argmax(y_pred_scratch_prob, axis=1)
y_true = y_test.flatten()

f1_scratch = f1_score(y_true, y_pred_scratch, average='macro')
print(f"Macro F1-score (from scratch): {f1_scratch:.4f}")

y_pred_keras_prob = keras_model.predict(x_test, batch_size=batch_size)
y_pred_keras = np.argmax(y_pred_keras_prob, axis=1)
f1_keras = f1_score(y_true, y_pred_keras, average='macro')
print(f"Macro F1-score (keras): {f1_keras:.4f}")

print("\nSample comparison [True, Scratch, Keras]:")
for i in range(10):
    print(f"{i}: {y_true[i]}  |  {y_pred_scratch[i]}  |  {y_pred_keras[i]}")

Macro F1-score (from scratch): 0.6827
 61/313 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

2025-05-29 12:17:21.418504: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-05-29 12:17:21.418747: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Macro F1-score (keras): 0.6827

Sample comparison [True, Scratch, Keras]:
0: 3  |  3  |  3
1: 8  |  8  |  8
2: 8  |  0  |  0
3: 0  |  8  |  8
4: 6  |  4  |  4
5: 6  |  6  |  6
6: 1  |  1  |  1
7: 6  |  6  |  6
8: 3  |  3  |  3
9: 1  |  1  |  1
